In [1]:
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import graphviz 
# might need to install -- need *both* python and system
# so use Anaconda (or pip) to install graphviz, and also https://graphviz.org/download/
# this was a non-trivial install on Mac (depends on Xcode command-line tools)
# `pip install graphviz` and `brew install graphviz`
import datetime as dt

%matplotlib inline
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier, export_graphviz
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import confusion_matrix, mean_squared_error, mean_absolute_error

In [83]:
bb = pd.read_csv("../../Data/1999_2021/aggragate_daily_data_2000to2020.csv", index_col = "date")[['influent_flow','maximum_demand']].loc['2008-01-01':'2017-12-31']

weather = pd.read_csv("../../Data/historic_weather_all_SP_daily.csv", index_col = "date")

Changing columns to (station, measurement)

In [3]:
### weather_tmp = weather.pivot(columns='station', values=['rain_mm', 'temp_max','temp_min'])



In [4]:
'''#really ugly. sorry.
weather_0 = weather_tmp.rename(columns={'rain_mm':'rain_mm_0', 'temp_max':'temp_max_0', 'temp_min':'temp_min_0'}).swaplevel(axis=1).sort_index(axis=1)
weather_7 = weather_tmp.shift(-7).rename(columns={'rain_mm':'rain_mm_7', 'temp_max':'temp_max_7', 'temp_min':'temp_min_7'}).swaplevel(axis=1).sort_index(axis=1)
weather_14 = weather_tmp.shift(-14).rename(columns={'rain_mm':'rain_mm_14', 'temp_max':'temp_max_14', 'temp_min':'temp_min_14'}).swaplevel(axis=1).sort_index(axis=1)
weather_30 = weather_tmp.shift(-30).rename(columns={'rain_mm':'rain_mm_30', 'temp_max':'temp_max_30', 'temp_min':'temp_min_30'}).swaplevel(axis=1).sort_index(axis=1)
'''

"#really ugly. sorry.\nweather_0 = weather_tmp.rename(columns={'rain_mm':'rain_mm_0', 'temp_max':'temp_max_0', 'temp_min':'temp_min_0'}).swaplevel(axis=1).sort_index(axis=1)\nweather_7 = weather_tmp.shift(-7).rename(columns={'rain_mm':'rain_mm_7', 'temp_max':'temp_max_7', 'temp_min':'temp_min_7'}).swaplevel(axis=1).sort_index(axis=1)\nweather_14 = weather_tmp.shift(-14).rename(columns={'rain_mm':'rain_mm_14', 'temp_max':'temp_max_14', 'temp_min':'temp_min_14'}).swaplevel(axis=1).sort_index(axis=1)\nweather_30 = weather_tmp.shift(-30).rename(columns={'rain_mm':'rain_mm_30', 'temp_max':'temp_max_30', 'temp_min':'temp_min_30'}).swaplevel(axis=1).sort_index(axis=1)\n"

In [80]:
rain = weather.pivot(columns='station', values=['rain_mm']).fillna(0)
rain_0 = rain.rename(columns={'rain_mm':'rain_mm_0'})
rain_7 = rain.shift(7).rename(columns={'rain_mm':'rain_mm_7'})
rain_14 = rain.shift(14).rename(columns={'rain_mm':'rain_mm_14'})
rain_30 = rain.shift(30).rename(columns={'rain_mm':'rain_mm_30'})

rain_agg = bb.join(rain_0).join(rain_7).join(rain_14).join(rain_30).dropna()


/Users/jzhu/opt/anaconda3/lib/python3.8/site-packages/pandas/core/reshape/merge.py:643: UserWarning: merging between different levels can give an unintended result (1 levels on the left,2 on the right)
  warnings.warn(msg, UserWarning)


In [53]:
rain_agg = bb

for i in range(0,8):
    rain_lag = rain.shift(i).rename(columns={'rain_mm':'rain_mm_' + str(i)})
    rain_agg = rain_agg.join(rain_lag).dropna()
    
rain_agg

/Users/jzhu/opt/anaconda3/lib/python3.8/site-packages/pandas/core/reshape/merge.py:643: UserWarning: merging between different levels can give an unintended result (1 levels on the left,2 on the right)
  warnings.warn(msg, UserWarning)
/Users/jzhu/opt/anaconda3/lib/python3.8/site-packages/pandas/core/reshape/merge.py:643: UserWarning: merging between different levels can give an unintended result (1 levels on the left,2 on the right)
  warnings.warn(msg, UserWarning)
/Users/jzhu/opt/anaconda3/lib/python3.8/site-packages/pandas/core/reshape/merge.py:643: UserWarning: merging between different levels can give an unintended result (1 levels on the left,2 on the right)
  warnings.warn(msg, UserWarning)
/Users/jzhu/opt/anaconda3/lib/python3.8/site-packages/pandas/core/reshape/merge.py:643: UserWarning: merging between different levels can give an unintended result (1 levels on the left,2 on the right)
  warnings.warn(msg, UserWarning)
/Users/jzhu/opt/anaconda3/lib/python3.8/site-packages/pa

,influent_flow,"(rain_mm_0, A701 SAO PAULO - MIRANTE)","(rain_mm_0, A705 BAURU)","(rain_mm_0, A706 CAMPOS DO JORDAO)","(rain_mm_0, A707 PRESIDENTE PRUDENTE)","(rain_mm_0, A708 FRANCA)","(rain_mm_0, A711 SAO CARLOS)","(rain_mm_0, A712 IGUAPE)","(rain_mm_0, A713 SOROCABA)","(rain_mm_0, A714 ITAPEVA)",...,"(rain_mm_7, A762 DRACENA)","(rain_mm_7, A763 MARILIA)","(rain_mm_7, A764 BEBDOURO)","(rain_mm_7, A765 BERTIOGA)","(rain_mm_7, A766 REGISTRO)","(rain_mm_7, A767 SAO SEBASTIAO)","(rain_mm_7, A768 TUPA)","(rain_mm_7, A769 CACHOEIRA PAULISTA)","(rain_mm_7, A770 SAO SIMAO)","(rain_mm_7, A771 SAO PAULO - INTERLAGOS)"
date,,,,,,,,,,,,,,,,,,,,,
2008-01-01,166.37,1.2,5.2,0.6,32.2,3.8,2.6,55.4,8.8,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2008-01-02,269.08,10.6,0.4,0.0,1.2,0.0,1.2,1.0,11.4,39.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2008-01-03,306.49,8.0,0.0,8.2,1.2,1.6,0.0,13.6,22.6,5.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2008-01-04,315.57,0.0,11.2,0.2,0.2,0.0,0.0,11.8,28.8,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2008-01-05,331.69,4.6,5.0,4.8,9.2,3.0,9.0,13.2,15.6,30.4,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2017-12-27,388.18,1.4,0.0,3.0,47.4,5.8,2.2,21.6,0.0,6.2,...,28.0,39.8,0.0,21.8,20.4,0.0,35.2,0.0,0.0,0.0
2017-12-28,462.12,3.2,1.4,1.2,56.8,16.4,0.8,2.8,3.8,8.6,...,6.4,1.6,2.4,11.6,30.0,7.4,6.4,18.0,0.0,0.0
2017-12-29,425.53,3.6,1.2,2.0,1.6,0.0,58.0,40.8,3.0,13.2,...,16.4,3.6,0.4,0.8,2.8,16.2,4.2,2.8,0.0,0.0


In [54]:
X_agg = rain_agg.drop('influent_flow', axis = 1)
y_agg = rain_agg.influent_flow
X_agg_train, X_agg_test, y_agg_train, y_agg_test = train_test_split(X_agg, y_agg, train_size = 0.8, random_state = 0)

In [27]:
'''rain_0 = rain_0.join(bb).dropna()
rain_7 = rain_7.join(bb).dropna()
rain_14 = rain_14.join(bb).dropna()
rain_30 = rain_30.join(bb).dropna()'''

'rain_0 = rain_0.join(bb).dropna()\nrain_7 = rain_7.join(bb).dropna()\nrain_14 = rain_14.join(bb).dropna()\nrain_30 = rain_30.join(bb).dropna()'

## This is all stations with lags of 7, 14, and 30. 

In [74]:
# All stations, all lags predicting energy generated at B. Bonita

random_forest_rain_agg = RandomForestRegressor(max_features = 3, random_state = 1230)

random_forest_rain_agg.fit(X_agg_train, y_agg_train)

pred_agg = random_forest_rain_agg.predict(X_agg_test)

Importance_agg = pd.DataFrame({'Importance':random_forest_rain_agg.feature_importances_*100}, 
                          index = X_agg.columns)

Importance_agg = Importance_agg.sort_values(by = 'Importance', axis = 0)

print("random forest MAE:", mean_absolute_error(y_agg_test, pred_agg))

random forest MAE: 139.9906517099863


In [75]:
Importance_agg = pd.DataFrame({'Importance':random_forest_rain_agg.feature_importances_*100}, 
                          index = X_agg.columns)

Importance_agg = Importance_agg.sort_values(by = 'Importance', axis = 0)

In [76]:
Importance_agg.sort_values('Importance',ascending=False).head(20)

,Importance
"(rain_mm_2, A729 VOTUPORANGA)",1.559102
"(rain_mm_2, A713 SOROCABA)",1.358634
"(rain_mm_2, A701 SAO PAULO - MIRANTE)",1.354225
"(rain_mm_1, A729 VOTUPORANGA)",1.017915
"(rain_mm_3, A726 PIRACICABA)",0.955849
"(rain_mm_2, A739 ITAPIRA)",0.955068
"(rain_mm_0, A729 VOTUPORANGA)",0.942061
"(rain_mm_3, A725 AVARE)",0.903304
"(rain_mm_3, A701 SAO PAULO - MIRANTE)",0.881598
"(rain_mm_2, A711 SAO CARLOS)",0.876293


In [78]:
Importance_agg.sort_values('Importance',ascending=False).tail(20)

,Importance
"(rain_mm_3, A744 BRAGANCA PAULISTA)",0.000102
"(rain_mm_4, A744 BRAGANCA PAULISTA)",0.000084
"(rain_mm_7, A744 BRAGANCA PAULISTA)",0.000003
"(rain_mm_4, A770 SAO SIMAO)",0.000000
"(rain_mm_5, A744 BRAGANCA PAULISTA)",0.000000
"(rain_mm_4, A771 SAO PAULO - INTERLAGOS)",0.000000
"(rain_mm_3, A770 SAO SIMAO)",0.000000
"(rain_mm_5, A770 SAO SIMAO)",0.000000
"(rain_mm_5, A771 SAO PAULO - INTERLAGOS)",0.000000
"(rain_mm_6, A770 SAO SIMAO)",0.000000


## All stations but only lagged 7 days

In [79]:
#Rain lagged 7 days predicting energy generated at B. Bonita
X_7 = rain_7.drop('influent_flow', axis = 1)
y_7 = rain_7.influent_flow
X_7_train, X_7_test, y_7_train, y_7_test = train_test_split(X_7, y_7, train_size = 0.8, random_state = 0)


random_forest_rain_7 = RandomForestRegressor(max_features = 42, random_state = 1)
random_forest_rain_7.fit(X_7_train, y_7_train)
pred_7 = random_forest_rain_7.predict(X_7_test)

Importance_7 = pd.DataFrame({'Importance':random_forest_rain_7.feature_importances_*100}, 
                          index = X_7.columns)

Importance_7 = Importance_7.sort_values(by = 'Importance', axis = 0)

print("random forest MAE:", mean_absolute_error(y_7_test, pred_7))

KeyError: 'influent_flow'

In [ ]:
Importance_7.sort_values('Importance', ascending=False).head(10)

In [ ]:
Importance_7.sort_values('Importance', ascending=False).tail(10)

## All stations lagged 14 days

In [ ]:
#Rain lagged 14 days predicting energy generated at B. Bonita

X_14 = rain_14.drop('influent_flow', axis = 1)
y_14 = rain_14.influent_flow
X_14_train, X_14_test, y_14_train, y_14_test = train_test_split(X_14, y_14, train_size = 0.8, random_state = 0)

random_forest_rain_14 = RandomForestRegressor(max_features = 10, random_state = 1)

random_forest_rain_14.fit(X_14_train, y_14_train)

pred_14 = random_forest_rain_14.predict(X_14_test)

Importance_14 = pd.DataFrame({'Importance':random_forest_rain_14.feature_importances_*100}, 
                          index = X_14.columns)

Importance_14 = Importance_14.sort_values(by = 'Importance', axis = 0)

print("random forest MAE:", mean_absolute_error(y_14_test, pred_14))


In [ ]:
Importance_14.sort_values('Importance',ascending=False).head(10)

In [ ]:
Importance_14.sort_values('Importance',ascending=False).tail(10)

## All stations lagged 30 days

In [ ]:
X_30 = rain_30.drop('influent_flow', axis = 1)
y_30 = rain_30.influent_flow
X_30_train, X_30_test, y_30_train, y_30_test = train_test_split(X_30, y_30, train_size = 0.8, random_state = 0)

random_forest_rain_30 = RandomForestRegressor(max_features = 10, random_state = 1)

random_forest_rain_30.fit(X_30_train, y_30_train)

pred_30 = random_forest_rain_30.predict(X_30_test)

Importance_30 = pd.DataFrame({'Importance':random_forest_rain_30.feature_importances_*100}, 
                          index = X_30.columns)

Importance_30 = Importance_30.sort_values(by = 'Importance', axis = 0)

print("random forest MAE:", mean_absolute_error(y_30_test, pred_30))


In [ ]:
Importance_30.sort_values('Importance',ascending=False).head(10)


In [ ]:
Importance_30.sort_values('Importance',ascending=False).tail(10)